<a href="https://colab.research.google.com/github/ryan-saloma/bike-market-summit/blob/main/Bike_Market_Summit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [103]:
%pip install -U googlemaps

import googlemaps as gm
import requests
import pandas as pd
import plotly
import plotly.express as px

gmaps = gm.Client(key='XXXXXXXXXXXXXXXXX')

# First get location of Summit, NJ
def get_coordinates_from_address(address, api_key):
    url = 'https://maps.googleapis.com/maps/api/geocode/json'
    params = {
        'address': address,
        'key': api_key
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        results = response.json().get('results')
        if results:
            location = results[0]['geometry']['location']
            return location['lat'], location['lng']
        else:
            print("No results found for the given city name.")
            return None
    else:
        print(f"Error: {response.status_code}")
        return None

summit_trek_coords = \
get_coordinates_from_address('348 Springfield Ave, Summit, NJ 07901', gmaps.key)

# Search for places nearby
# TODO: reduce search radius to 10k
nearby_bike_stores = gmaps.places_nearby(location=summit_trek_coords, \
                                         keyword= 'bike store',radius=20000, \
                                         type='bicycle_store')
# Get results as a list of dicts
results = nearby_bike_stores['results']

# Perform list comprehension to extract keys of interest
keys_of_interest = ['name', 'rating', 'user_ratings_total', 'geometry']
stores_df = pd.DataFrame(results)
stores_df

# Get coords of competing stores
coords_competition = pd.DataFrame()
coords_competition['Name'] = stores_df['name']
coords_competition['Latitude'] = stores_df['geometry'].apply(lambda x: x['location']['lat'])
coords_competition['Longitude'] = stores_df['geometry'].apply(lambda x: x['location']['lng'])
coords_competition['Rating'] = stores_df['rating']
coords_competition['Number of Ratings'] = stores_df['user_ratings_total']
coords_competition

coords_competition.dropna(
    axis=0,
    how='any',
    subset=None,
    inplace=True
)

color_scale = [(0, 'orange'), (1,'red')]

fig = px.scatter_mapbox(coords_competition,
                        lat="Latitude",
                        lon="Longitude",
                        hover_name="Name",
                        hover_data=["Name", "Rating"],
                        color="Rating",
                        color_continuous_scale=color_scale,
                        size="Number of Ratings",
                        zoom=8,
                        height=800,
                        width=800)

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()
plotly.io.write_html(fig, "competition_map.html")
